# Organización de los datos del acelerómetro

In [ ]:
import csv
import numpy as np 
import pandas as pd

def seperate_pid_data():
	acc_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Modelos/data/all_accelerometer_data_pids_13.csv")
	pids = list(set(acc_data['pid']))
	for pid in pids:
		df = acc_data.loc[acc_data['pid'] == pid]
		df.to_csv("/content/drive/MyDrive/Colab Notebooks/Modelos/data/clean acc/" + str(pid) + ".csv", encoding='utf-8')
		print(str(pid)+".csv created")
	print(pids)

	return pids

In [ ]:
seperate_pid_data()

# Agrupar lecturas del acelerómetro por segundo

In [ ]:
def createLecturesPerSecond(maxNumber, file, test = False, testNumber = 1):
  acc = pd.read_csv(file)
  counter = 0
  prevSec = acc.iloc[0]
  prevSec = prevSec['time'] // 1000  #pasarlo a segundos
  accSecTemp = list()
  accSec = list()
  rows = len(acc) if test == False else testNumber

  for i in range(0, rows):
    sample = acc.iloc[i]
    currSec = sample['time']//1000  #esta en milisegundos, pa pasarlo a segundos

    if i != 0:
      prevSec = acc.loc[i-1, 'time']//1000 #pasarlo a segundos

    if currSec == prevSec and counter < maxNumber:
      accSecTemp.append([sample['time']//1000, sample['x'], sample['y'], sample['z']])
      counter += 1
      if counter == maxNumber:
        accSec.append(np.array(accSecTemp))
        accSecTemp.clear()
    
    elif currSec > prevSec:
      accSecTemp.clear()
      accSecTemp.append([sample['time']//1000, sample['x'], sample['y'], sample['z']])
      counter = 1
  
  accSec = np.array(accSec)
  np.set_printoptions(formatter={'all':lambda x: str(x)})
  return accSec

# Creación de la ventana de tiempo

In [ ]:
import pickle
def createWindow(size, lecturesPerSecond, fileName, testeo = False, rowsNumber = 0):
  lectPerSec = createLecturesPerSecond(maxNumber=lecturesPerSecond, file=fileName, test=testeo, testNumber=rowsNumber)
  tempWindow = list()
  window = list()
  rows = len(lectPerSec)
  i = 0
  while i + size < rows:
    if int(lectPerSec[i + size - 1][0][0]) == int(lectPerSec[i][0][0]) + size - 1:
      print("entra")
      for j in range(i, i + size):
        for k in range(0, len(lectPerSec[j])):
          tempWindow.append(lectPerSec[j][k])
      window.append(np.array(tempWindow))
      tempWindow.clear()
    i+=1
  window = np.array(window)
  filename = "/content/drive/MyDrive/Colab Notebooks/Modelos/data/pickles/" + str(fileName.split('/')[-1].split('.')[0]) +  ".pkl"
  outfile = open(filename, 'wb')
  pickle.dump(window, outfile)
  outfile.close()
  return window

In [ ]:
pids = ['BU4707', 'PC6771', 'JB3156', 'JR8022', 'DK3500', 'BK7610', 'HV0618', 'MJ8002', 'DC6359', 'CC6740', 'SF3079', 'MC7070', 'SA0297']
for pid in pids:
  file = "/content/drive/MyDrive/Colab Notebooks/Modelos/data/clean acc/" + pid + ".csv"
  createWindow(10, 20, file)

# Categorización de las etiquetas de salida

In [ ]:
def getTACLabels(file, threshold = 0.08):
  labels = list()
  tac = pd.read_csv(file)

  rows = len(tac)
  for i in range(0, rows):
    if tac.loc[i, 'TAC_Reading'] >= threshold:
      labels.append(1)
    else:
      labels.append(0)
  
  tac['y'] = labels
  del tac['TAC_Reading']
  tac.to_csv("/content/drive/MyDrive/Colab Notebooks/Modelos/data/clean_tac/Labels/" + str(file.split('/')[-1].split('.')[0]) + "label.csv")

  return str(file.split('/')[-1].split('.')[0]) + "label.csv"

In [ ]:
for pid in pids:
  getTACLabels("/content/drive/MyDrive/Colab Notebooks/Modelos/data/clean_tac/" + pid + "_clean_TAC.csv")